In [1]:
%matplotlib inline

In [2]:
from keras.preprocessing import image
from matplotlib import pyplot as plt
import numpy as np

Using TensorFlow backend.


# Loading the training data

In [3]:
train_batches = image.ImageDataGenerator().flow_from_directory(
    '/data/dog_breeds/train', 
    target_size=(224,224),
    class_mode='categorical', 
    shuffle=True, 
    batch_size=8
)

valid_batches = image.ImageDataGenerator().flow_from_directory(
    '/data/dog_breeds/valid', 
    target_size=(224,224),
    class_mode='categorical', 
    shuffle=True, 
    batch_size=8
)

Found 9422 images belonging to 121 classes.
Found 800 images belonging to 121 classes.


# Loading VGG

In [10]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Nadam

In [5]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1,1,3))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def ConvBlock(model, layers, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    
def BuildVGG():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(224,224,3)))
    ConvBlock(model, 2, 64)
    ConvBlock(model, 2, 128)
    ConvBlock(model, 3, 256)
    ConvBlock(model, 3, 512)
    ConvBlock(model, 3, 512)

    model.add(Flatten(name='flatten'))
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))

    model.load_weights('/data/trained_models/vgg16_tf.h5')
    model.compile(optimizer=Nadam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model



# Full training

In [11]:
cd_model = BuildVGG()
cd_model.pop()
cd_model.pop()
cd_model.pop()
cd_model.pop()
cd_model.pop()
cd_model.add(Dense(4096, activation='relu'))
cd_model.add(BatchNormalization())
cd_model.add(Dropout(0.3))
cd_model.add(Dense(4096, activation='relu'))
cd_model.add(BatchNormalization())
cd_model.add(Dropout(0.3))
cd_model.add(Dense(121, activation='softmax'))

opt = Nadam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
cd_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# find the index of the flatten layer
original_layers = cd_model.layers
flatten = [x for x in original_layers if x.name == 'flatten'][0]
flatten_idx = original_layers.index(flatten)

In [14]:
def set_trainable_layers_after(n, model):
    for i, layer in enumerate(cd_model.layers):
        layer.trainable = i > n

In [15]:
train_batches_augmented = image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True
).flow_from_directory(
    '/data/dog_breeds/train', 
    target_size=(224,224),
    class_mode='categorical', 
    shuffle=True, 
    batch_size=8
)

Found 9422 images belonging to 121 classes.


In [16]:
set_trainable_layers_after(flatten_idx, cd_model)

In [17]:
cd_model.fit_generator(train_batches_augmented, 
                 steps_per_epoch=train_batches.samples/train_batches.batch_size,
                 epochs=4, 
                 validation_data=valid_batches, 
                 validation_steps=valid_batches.samples/valid_batches.batch_size
                )

Epoch 1/4
1178/1177 [==============================] - 235s - loss: 5.7907 - acc: 0.0103 - val_loss: 5.6328 - val_acc: 0.0138
Epoch 2/4
1178/1177 [==============================] - 232s - loss: 5.6271 - acc: 0.0119 - val_loss: 15.7826 - val_acc: 0.0112
Epoch 3/4
1178/1177 [==============================] - 227s - loss: 5.5970 - acc: 0.0126 - val_loss: 8.8515 - val_acc: 0.0063
Epoch 4/4
1178/1177 [==============================] - 229s - loss: 5.5854 - acc: 0.0133 - val_loss: 5.3614 - val_acc: 0.0187


In [22]:
for idx in range(flatten_idx-1, -1, -1):
    set_trainable_layers_after(idx, cd_model)
    cd_model.fit_generator(train_batches_augmented, 
                     steps_per_epoch=train_batches.samples/train_batches.batch_size,
                     epochs=1, 
                     validation_data=valid_batches, 
                     validation_steps=valid_batches.samples/valid_batches.batch_size
                    )

Epoch 1/1
1178/1177 [==============================] - 228s - loss: 5.5268 - acc: 0.0145 - val_loss: 6.7538 - val_acc: 0.0175
Epoch 1/1
1178/1177 [==============================] - 229s - loss: 5.4958 - acc: 0.0158 - val_loss: 5.3313 - val_acc: 0.0325
Epoch 1/1
1178/1177 [==============================] - 228s - loss: 5.4974 - acc: 0.0144 - val_loss: 5.3517 - val_acc: 0.0250
Epoch 1/1
1178/1177 [==============================] - 229s - loss: 5.4704 - acc: 0.0140 - val_loss: 6.4834 - val_acc: 0.0163
Epoch 1/1
1178/1177 [==============================] - 229s - loss: 5.4261 - acc: 0.0163 - val_loss: 5.8758 - val_acc: 0.0275
Epoch 1/1
1178/1177 [==============================] - 231s - loss: 5.4319 - acc: 0.0129 - val_loss: 5.7108 - val_acc: 0.0112
Epoch 1/1
1178/1177 [==============================] - 232s - loss: 5.3989 - acc: 0.0135 - val_loss: 5.2030 - val_acc: 0.0213
Epoch 1/1
1178/1177 [==============================] - 226s - loss: 5.3745 - acc: 0.0171 - val_loss: 6.3330 - val_acc:

In [23]:
cd_model.save_weights('/data/trained_models/dog_breeds_fulltrain_v1.h5')